In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import tensorflow as tf
%matplotlib inline

In [23]:
# Import our input dataset
charity_df = pd.read_csv('charity_data.csv')

## Drop ID columns

In [24]:
name=charity_df.NAME
ein=charity_df.EIN
charity_df=charity_df.drop(["EIN","NAME"],axis=1)

## Bin column values

In [4]:
# Generate categorical variable list
cats = charity_df.dtypes[charity_df.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
charity_df[cats].nunique()

APPLICATION_TYPE          17
AFFILIATION                6
CLASSIFICATION            71
USE_CASE                   5
ORGANIZATION               4
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
dtype: int64

In [5]:
x=charity_df.CLASSIFICATION.value_counts()
x.head(10)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
Name: CLASSIFICATION, dtype: int64

In [6]:
# Determine which values to replace
replace_class = list(x[5:].index)

# Replace in DataFrame
for c in replace_class:
    charity_df.CLASSIFICATION = charity_df.CLASSIFICATION.replace(c,"Other")

# Check to make sure binning was successful
charity_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [7]:
x=charity_df.APPLICATION_TYPE.value_counts()
x

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Determine which values to replace
replace_app = list(x[7:].index)

# Replace in DataFrame
for a in replace_class:
    charity_df.APPLICATION_TYPE = charity_df.APPLICATION_TYPE.replace(a,"Other")

# Check to make sure binning was successful
charity_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

## Encode variables

In [9]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(charity_df[cats]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(cats)
encode_df.head()

,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
# Merge one-hot encoded features and drop the originals
charity_df = charity_df.merge(encode_df,left_index=True, right_index=True)
charity_df = charity_df.drop(cats,1)
charity_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


## Remove Target

In [11]:
# Remove target from features data
y = charity_df.IS_SUCCESSFUL
X = charity_df.drop('IS_SUCCESSFUL',axis=1)

## Scaling

In [12]:
# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Model V1

In [13]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  10
hidden_nodes_layer2 = 5


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, 
                             input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                520       
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 55        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 6         
Total params: 581
Trainable params: 581
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss1, model_accuracy1 = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss1}, Accuracy: {model_accuracy1}")

Train on 25724 samples
Epoch 1/50
25724/25724 [==============================] - 2s 81us/sample - loss: 0.6254 - accuracy: 0.6827
Epoch 2/50
25724/25724 [==============================] - 1s 54us/sample - loss: 0.5727 - accuracy: 0.7202
Epoch 3/50
25724/25724 [==============================] - 1s 43us/sample - loss: 0.5603 - accuracy: 0.7249
Epoch 4/50
25724/25724 [==============================] - 1s 48us/sample - loss: 0.5567 - accuracy: 0.7282
Epoch 5/50
25724/25724 [==============================] - 1s 45us/sample - loss: 0.5548 - accuracy: 0.7295s - loss: 0
Epoch 6/50
25724/25724 [==============================] - 1s 44us/sample - loss: 0.5536 - accuracy: 0.7294s - l - ETA: 0s - loss: 0.5559 - ac
Epoch 7/50
25724/25724 [==============================] - 1s 44us/sample - loss: 0.5525 - accuracy: 0.7306
Epoch 8/50
25724/25724 [==============================] - 1s 44us/sample - loss: 0.5515 - accuracy: 0.7304s - l
Epoch 9/50
25724/25724 [==============================] - 1s 44us/samp

## Model V2 (More Epochs)

In [15]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  10
hidden_nodes_layer2 = 5


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, 
                             input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
nn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 10)                520       
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 55        
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 6         
Total params: 581
Trainable params: 581
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

# Evaluate the model using the test data
model_loss2, model_accuracy2 = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss2}, Accuracy: {model_accuracy2}")

Train on 25724 samples
Epoch 1/100
25724/25724 [==============================] - 2s 92us/sample - loss: 0.6212 - accuracy: 0.6950
Epoch 2/100
25724/25724 [==============================] - 1s 58us/sample - loss: 0.5767 - accuracy: 0.7222
Epoch 3/100
25724/25724 [==============================] - 1s 54us/sample - loss: 0.5648 - accuracy: 0.7260
Epoch 4/100
25724/25724 [==============================] - 1s 54us/sample - loss: 0.5595 - accuracy: 0.7285
Epoch 5/100
25724/25724 [==============================] - 1s 55us/sample - loss: 0.5570 - accuracy: 0.7291
Epoch 6/100
25724/25724 [==============================] - 1s 53us/sample - loss: 0.5551 - accuracy: 0.7305
Epoch 7/100
25724/25724 [==============================] - 1s 49us/sample - loss: 0.5539 - accuracy: 0.7308
Epoch 8/100
25724/25724 [==============================] - 1s 44us/sample - loss: 0.5532 - accuracy: 0.7310s - los
Epoch 9/100
25724/25724 [==============================] - 1s 48us/sample - loss: 0.5523 - accuracy: 0.732

## Model V3 (More Nodes)

In [17]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  25
hidden_nodes_layer2 = 10


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, 
                             input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
nn.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 25)                1300      
_________________________________________________________________
dense_7 (Dense)              (None, 10)                260       
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 11        
Total params: 1,571
Trainable params: 1,571
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

# Evaluate the model using the test data
model_loss3, model_accuracy3 = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss3}, Accuracy: {model_accuracy3}")

Train on 25724 samples
Epoch 1/100
25724/25724 [==============================] - 2s 76us/sample - loss: 0.5851 - accuracy: 0.7083
Epoch 2/100
25724/25724 [==============================] - 1s 48us/sample - loss: 0.5576 - accuracy: 0.7269
Epoch 3/100
25724/25724 [==============================] - 1s 57us/sample - loss: 0.5543 - accuracy: 0.7311
Epoch 4/100
25724/25724 [==============================] - 2s 78us/sample - loss: 0.5524 - accuracy: 0.7322
Epoch 5/100
25724/25724 [==============================] - 2s 81us/sample - loss: 0.5505 - accuracy: 0.7327
Epoch 6/100
25724/25724 [==============================] - 2s 74us/sample - loss: 0.5497 - accuracy: 0.7335
Epoch 7/100
25724/25724 [==============================] - 2s 62us/sample - loss: 0.5491 - accuracy: 0.7334
Epoch 8/100
25724/25724 [==============================] - 2s 60us/sample - loss: 0.5483 - accuracy: 0.7337
Epoch 9/100
25724/25724 [==============================] - 1s 57us/sample - loss: 0.5480 - accuracy: 0.7330
Epoch

## Model V4 (More Layers)

In [19]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  40
hidden_nodes_layer2 = 20
hidden_nodes_layer3 = 10


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, 
                             input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
nn.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 40)                2080      
_________________________________________________________________
dense_10 (Dense)             (None, 20)                820       
_________________________________________________________________
dense_11 (Dense)             (None, 10)                210       
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 11        
Total params: 3,121
Trainable params: 3,121
Non-trainable params: 0
_________________________________________________________________


In [20]:
# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

# Evaluate the model using the test data
model_loss4, model_accuracy4 = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss4}, Accuracy: {model_accuracy4}")

Train on 25724 samples
Epoch 1/100
25724/25724 [==============================] - 3s 101us/sample - loss: 0.5804 - accuracy: 0.7166
Epoch 2/100
25724/25724 [==============================] - 2s 64us/sample - loss: 0.5540 - accuracy: 0.7311
Epoch 3/100
25724/25724 [==============================] - 2s 65us/sample - loss: 0.5518 - accuracy: 0.7317
Epoch 4/100
25724/25724 [==============================] - 2s 65us/sample - loss: 0.5502 - accuracy: 0.7321
Epoch 5/100
25724/25724 [==============================] - 2s 66us/sample - loss: 0.5487 - accuracy: 0.7334
Epoch 6/100
25724/25724 [==============================] - 2s 66us/sample - loss: 0.5479 - accuracy: 0.7345
Epoch 7/100
25724/25724 [==============================] - 2s 67us/sample - loss: 0.5472 - accuracy: 0.7348
Epoch 8/100
25724/25724 [==============================] - 2s 67us/sample - loss: 0.5467 - accuracy: 0.7344
Epoch 9/100
25724/25724 [==============================] - 2s 65us/sample - loss: 0.5451 - accuracy: 0.7360
Epoc

In [21]:
print(f"Verion 1: {model_accuracy1} \nVerion 2: {model_accuracy2} \nVerion 3: {model_accuracy3} \nVerion 4: {model_accuracy4}")

Verion 1: 0.7250145673751831 
Verion 2: 0.7237317562103271 
Verion 3: 0.7259474992752075 
Verion 4: 0.7255976796150208


In [22]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=256, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Random forest predictive accuracy: 0.708
